Instructions for installing GDAL for Ubuntu: https://mothergeo-py.readthedocs.io/en/latest/development/how-to/gdal-ubuntu-pkg.html

The following are the steps detailed in the above article:

sudo add-apt-repository ppa:ubuntugis/ppa && sudo apt-get update
sudo apt-get update
sudo apt-get install gdal-bin
sudo apt-get install libgdal-dev
export CPLUS_INCLUDE_PATH=/usr/include/gdal
export C_INCLUDE_PATH=/usr/include/gdal
pip install GDAL


I also had to run 'pip install wheel' before installing GDAL

In [7]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr

In [8]:
dataset_path = '/dataset/Copy of Blore_Clean.tif'
ds = gdal.Open(dataset_path)

In [9]:
# transformation described here: https://arijolma.org/Geo-GDAL/2.0/classGeo_1_1GDAL_1_1GeoTransform.html#details

# NOTE: pixel coordinate (0, 0) is the top left pixel of the image

def transform_pixel_coords_to_projection_coords(gt, column, row):
    a,b,c,d,e,f = gt
    x = a + column * b + row * c
    y = d + column * e + row * f
    
    return x,y

In [10]:
# Transforms pixel coordinate (column, row) of dataset to WGS84 geographic coordinates

def transform_image_pixel_coords_to_WGS84_geographic_coords(ds, column, row):
    # Input coordinate system: specified in input tif file
    InSR = osr.SpatialReference()
    InSR.ImportFromWkt(ds.GetProjectionRef())
    
    # Output coordinate system: WGS84/Geographic
    OutSR = osr.SpatialReference()
    OutSR.ImportFromEPSG(4326)
    
    # get pixel width and height of image
    width = ds.RasterXSize 
    height = ds.RasterYSize 
    
    # transformation from pixel coordinates to projection coordinates
    gt = ds.GetGeoTransform()
    
    # Transform top left pixel coordinate (0, 0) to projection coordinates
    x,y = transform_pixel_coords_to_projection_coords(gt, 0, 0)
    
    # transformation from projection coordinates to WGS84 geographic coordinates
    transform = osr.CoordinateTransformation(InSR, OutSR)
    
    # geographic latitude and longitude of point
    latlong = transform.TransformPoint(x,y)
    
    return latlong

In [11]:
latlong = transform_image_pixel_coords_to_WGS84_geographic_coords(ds, 0, 0)

print(latlong)

(12.86308254761559, 77.5151947517078, 0.0)
